In [1]:
%%time
import pandas as pd
import numpy as np
import time
import pulp 

job_num = 10
processing_df= pd.read_csv('Processing_time.csv')
sequence_df = pd.read_csv('Machine_sequence.csv')

processing_df = processing_df.drop(['Job'],axis=1)
sequence_df = sequence_df.drop(['Job'],axis=1)


pt = processing_df.values
mo_0 = sequence_df.values
mo = mo_0-1

dv = pulp.LpVariable.dicts("start_time",
                                     ((i, j) for i in range(job_num) for j in range(job_num)),
                                     lowBound=0,
                                     cat='Continuous')

b =  pulp.LpVariable.dicts("binary_var",
                                     ((i, j,o) for i in range(job_num) for j in range(job_num) for o in range(job_num) if j<o),
                                     lowBound=0,
                                     cat='Binary')

Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')

model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)
model += Cmax

for j in range(job_num):
    for i in range(job_num-1):
       I = mo[j,i]
       K = mo[j,i+1]
       J = j
       model += (dv[K,J] - dv[I,J]) >= pt[J][list(mo[J]).index(I)]

for j in range(job_num):
    for i in range(job_num):
       I = mo[j,i]
       J = j
       model += (Cmax - dv[I,J]) >= pt[J][list(mo[J]).index(I)]

for i in range(job_num):
    for j in range(job_num):
        for l in range(job_num):
            if j<l:
                I=i
                L = l
                J = j
                model += (dv[I,J] - dv[I,L]) >= (pt[L][list(mo[L]).index(I)] - 1000*(1-b[i,J,L]))
                model += (dv[I,L] - dv[I,J]) >= (pt[J][list(mo[J]).index(I)] - 1000*(b[i,J,L]))
       
model.solve()
pulp.LpStatus[model.status]     

for var in dv:
    var_value = dv[var].varValue
    print( var[0],'-',var[1] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print( pulp.LpStatus[model.status] )

0 - 0 129.0
0 - 1 200.0
0 - 2 364.0
0 - 3 59.0
0 - 4 725.0
0 - 5 228.0
0 - 6 143.0
0 - 7 326.0
0 - 8 746.0
0 - 9 0.0
1 - 0 339.0
1 - 1 256.0
1 - 2 817.0
1 - 3 368.0
1 - 4 450.0
1 - 5 646.0
1 - 6 372.0
1 - 7 572.0
1 - 8 526.0
1 - 9 174.0
2 - 0 400.0
2 - 1 390.0
2 - 2 661.0
2 - 3 418.0
2 - 4 746.0
2 - 5 11.0
2 - 6 490.0
2 - 7 0.0
2 - 8 295.0
2 - 9 325.0
3 - 0 424.0
3 - 1 605.0
3 - 2 0.0
3 - 3 748.0
3 - 4 627.0
3 - 5 326.0
3 - 6 94.0
3 - 7 228.0
3 - 8 143.0
3 - 9 519.0
4 - 0 158.0
4 - 1 712.0
4 - 2 172.0
4 - 3 526.0
4 - 4 626.0
4 - 5 200.0
4 - 6 484.0
4 - 7 646.0
4 - 8 701.0
4 - 9 68.0
5 - 0 162.0
5 - 1 497.0
5 - 2 579.0
5 - 3 664.0
5 - 4 306.0
5 - 5 55.0
5 - 6 450.0
5 - 7 364.0
5 - 8 789.0
5 - 9 257.0
6 - 0 143.0
6 - 1 643.0
6 - 2 826.0
6 - 3 164.0
6 - 4 275.0
6 - 5 123.0
6 - 6 584.0
6 - 7 445.0
6 - 8 776.0
6 - 9 304.0
7 - 0 30.0
7 - 1 112.0
7 - 2 822.0
7 - 3 0.0
7 - 4 200.0
7 - 5 425.0
7 - 6 771.0
7 - 7 255.0
7 - 8 326.0
7 - 9 787.0
8 - 0 271.0
8 - 1 419.0
8 - 2 762.0
8 - 3 500.0
8 - 4 

In [24]:
'''--------plot gantt chart-------'''
import pandas as pd
from chart_studio import plotly as py
from plotly.offline import iplot
import plotly.figure_factory as ff
import datetime

m_keys=[j for j in range(10)]
j_keys=[j for j in range(10)]
key_count={key:0 for key in j_keys}
j_record={}


for var in dv:
    var_value = dv[var].varValue
    
    start_time=str(datetime.timedelta(seconds=var_value)) # convert seconds to hours, minutes and seconds
    end_time=str(datetime.timedelta(seconds=var_value+pt[var[1]][list(mo[var[1]]).index(var[0])]))
        
    j_record[(var[1],var[0])]=[start_time,end_time]
    
    key_count[i]=key_count[i]+1
        

df=[]
for m in m_keys:
    for j in j_keys:
        df.append(dict(Task='Machine %s'%(m+1), Start='2023-10-15 %s'%(str(j_record[(j,m)][0])), Finish='2023-10-15 %s'%(str(j_record[(j,m)][1])),Resource='Job %s'%(j+1)))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Job shop Schedule')
iplot(fig, filename='GA_job_shop_scheduling')

In [17]:
j_record.keys()

dict_keys([(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (9, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6), (8, 6), (9, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9)])